In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset, Dataset
from torchvision import datasets, transforms, models
import timm
import numpy as np
from matplotlib import pyplot as plt

In [2]:
import sys
import os

In [1]:
from base_trainer import Trainer
from model import SimpleCNN

In [15]:
import json

In [5]:

NUM_CLASSES = 10           # CIFAR-10


In [6]:
# Загружаем CIFAR-10
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
train_dataset = datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
class_indices = {class_idx: [] for class_idx in range(NUM_CLASSES)}
for idx, (_, label) in enumerate(train_dataset):
    class_indices[label].append(idx)


EPOCHS = 20
learning_rate = 5e-4

val_dataloader = DataLoader(test_dataset, batch_size=128, shuffle=True)

In [9]:


val_dict = {}
for p in [0.01, 0.1, 0.2]:

    f1 = 0.0
    for _ in range(5):

        # Разделяем данные
        initial_indices = []
        pool_indices = []

        for class_idx, indices in class_indices.items():
            np.random.shuffle(indices)  # Перемешиваем индексы внутри класса
            split_idx = int(len(indices) * p)  # Вычисляем границу разделения
            initial_indices.extend(indices[:split_idx])
        
        initial_train_dataset = Subset(train_dataset, initial_indices)
        print(f"Размер initial_train_dataset: {len(initial_train_dataset)}")
        train_dataloader = DataLoader(initial_train_dataset, batch_size=128, shuffle=True)

        model = SimpleCNN(NUM_CLASSES).to(DEVICE)
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate )
        criterion = torch.nn.CrossEntropyLoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
        trainer = Trainer(model = model,
                          optimizer=optimizer,
                          pool_loader=None,
                          criterion=criterion ,
                          train_loader=train_dataloader,
                          val_loader = val_dataloader,
                          device=DEVICE,
                          scheduler=scheduler)
        trainer.fit(EPOCHS)
        f1 += max(trainer.val_f1)
        del optimizer 
        del criterion
        del scheduler
        del model
        del trainer
    f1 /= 5
    val_dict[p] = f1
    torch.cuda.empty_cache()
    print(f"f1 for random init: p = {p}, f1 = {val_dict[p]}")





Размер initial_train_dataset: 500


Validating: 100%|██████████| 79/79 [00:01<00:00, 44.31it/s]


Validation Loss: 2.2639, Accuracy: 0.1132, F1 Score: 0.0417
Epoch 1/20 - Train Loss: 2.3056, Val Loss: 2.2639, Acc: 0.1132, F1: 0.0417


Validating: 100%|██████████| 79/79 [00:01<00:00, 43.81it/s]


Validation Loss: 2.2036, Accuracy: 0.2498, F1 Score: 0.2114
Epoch 2/20 - Train Loss: 2.2323, Val Loss: 2.2036, Acc: 0.2498, F1: 0.2114


...

Validation Loss: 1.0625, Accuracy: 0.6288, F1 Score: 0.6254
Epoch 18/20 - Train Loss: 0.7649, Val Loss: 1.0625, Acc: 0.6288, F1: 0.6254


Validating: 100%|██████████| 79/79 [00:02<00:00, 34.07it/s]


Validation Loss: 1.0585, Accuracy: 0.6324, F1 Score: 0.6326
Epoch 19/20 - Train Loss: 0.7408, Val Loss: 1.0585, Acc: 0.6324, F1: 0.6326


Validating: 100%|██████████| 79/79 [00:02<00:00, 37.37it/s]


Validation Loss: 1.0437, Accuracy: 0.6361, F1 Score: 0.6357
Epoch 20/20 - Train Loss: 0.7127, Val Loss: 1.0437, Acc: 0.6361, F1: 0.6357
f1 for random init: p = 0.2, f1 = 0.6282874119768651


In [10]:
val_dict

{0.01: 0.3620172704387155, 0.1: 0.5649947488309035, 0.2: 0.6282874119768651}

In [10]:
train_dataset.__len__()

50000

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)
model = SimpleCNN(NUM_CLASSES).to(DEVICE)
optimizer = optim.AdamW(model.parameters(), lr=5e-4 )
criterion = torch.nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
trainer = Trainer(model = model,
                  optimizer=optimizer,
                  pool_loader=None,
                  criterion=criterion ,
                  train_loader=train_dataloader,
                  val_loader = val_dataloader,
                  device=DEVICE,
                  scheduler=scheduler)
trainer.fit(EPOCHS)


Validating: 100%|██████████| 79/79 [00:02<00:00, 37.80it/s]


Validation Loss: 1.2716, Accuracy: 0.5568, F1 Score: 0.5503
Epoch 1/20 - Train Loss: 1.5428, Val Loss: 1.2716, Acc: 0.5568, F1: 0.5503


Validating: 100%|██████████| 79/79 [00:02<00:00, 34.29it/s]


Validation Loss: 1.1419, Accuracy: 0.6048, F1 Score: 0.6017
Epoch 2/20 - Train Loss: 1.2206, Val Loss: 1.1419, Acc: 0.6048, F1: 0.6017


Validating: 100%|██████████| 79/79 [00:02<00:00, 36.50it/s]


...

Validation Loss: 0.7418, Accuracy: 0.7499, F1 Score: 0.7475
Epoch 18/20 - Train Loss: 0.5471, Val Loss: 0.7418, Acc: 0.7499, F1: 0.7475


Validating: 100%|██████████| 79/79 [00:02<00:00, 38.58it/s]


Validation Loss: 0.7632, Accuracy: 0.7422, F1 Score: 0.7386
Epoch 19/20 - Train Loss: 0.5277, Val Loss: 0.7632, Acc: 0.7422, F1: 0.7386


Validating: 100%|██████████| 79/79 [00:02<00:00, 36.39it/s]

Validation Loss: 0.7492, Accuracy: 0.7469, F1 Score: 0.7442
Epoch 20/20 - Train Loss: 0.5129, Val Loss: 0.7492, Acc: 0.7469, F1: 0.7442


In [12]:
val_dict[1.0] = max(trainer.val_f1)

In [14]:
wrapped_dict = {"random": val_dict}
wrapped_dict

{'random': {0.01: 0.3620172704387155,
  0.1: 0.5649947488309035,
  0.2: 0.6282874119768651,
  1.0: 0.747540465671377}}

In [17]:
# Сохраняем в JSON
with open("vals.json", "w", encoding="utf-8") as file:
    json.dump(wrapped_dict, file, ensure_ascii=False, indent=4)